In [ ]:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install lmdb pillow torchvision nltk natsort fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=3e0d202fce5c2177e44c225d74136f743df7bdea7fc26ee5f9d1b7c9fb71f915
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

src_df = pd.read_csv('../input/sberocr/Train/train.csv')

train, test = train_test_split(src_df, test_size=0.2, random_state=0)

In [ ]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark

In [ ]:
import json
import cv2
import numpy as np

In [ ]:
!rm -rf train/
!mkdir -p train/images

In [ ]:
dir = 'train/'

for index, row in train.iterrows():
  data = json.loads(row['output'])[0]
  img = cv2.imread('../input/sberocr/Train/'+row['image_path'])
  for i, line in enumerate(data):
    label, left, top, width, height = (lambda label, left, top, width, height, shape: (label, left, top, width, height))(**line)
    if '\n' in label:
      continue
    im_width, im_height, channels = img.shape
    x, y = round(im_height*left), round(im_width*top)
    w, h = round(im_height*width), round(im_width*height)
    # print(x, y, w, h)
    cropped_image = img[y:y+h, x:x+w]
    im_name = f'images/{index}_{i}.jpg'
    try:
      cv2.imwrite(dir+im_name, cropped_image)
    except:
      continue
    with open(dir+"gt.txt", "a") as file:
      file.write(f'{im_name}\t{label}\n')

In [ ]:
!python deep-text-recognition-benchmark/create_lmdb_dataset.py --inputPath train/ --gtFile train/gt.txt --outputPath result/train/

In [ ]:
!cp ../input/utils-patch4/utils.py deep-text-recognition-benchmark/utils.py
!cp ../input/dataset-patch/dataset.py deep-text-recognition-benchmark/dataset.py

In [ ]:
!cat deep-text-recognition-benchmark/utils.py

In [5]:
!CUDA_VISIBLE_DEVICES=0 python deep-text-recognition-benchmark/train.py \
--train_data result/train --valid_data result/test \
--select_data / --batch_ratio 1 --valInterval 100 --rgb --input_channel 3 --num_iter 30000 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--character "0123456789,.?!:&*()%-=+ abcdefghijklmnopqrstuvwxyzабвгдеёжзийклмнопрстуфхцчшщъыьэюяABCDEFGHIJKLMNOPQRSTUVWXYZАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ" #ABCDEFGHIJKLMNOPQRSTUVWXYZАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: result/train
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    result/train	 dataset: /
sub-directory:	/.	 num samples: 28800
num total samples of /: 28800 x 1.0 (total_data_usage_ratio) = 28800
num samples of / per batch: 192 x 1.0 (batch_ratio) = 192
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness

In [4]:
!ls -al ./saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111

total 389492
drwxr-xr-x 2 root root      4096 Oct 30 03:27 .
drwxr-xr-x 7 root root      4096 Oct 30 03:27 ..
-rw-r--r-- 1 root root 199271205 Oct 30 03:27 best_accuracy.pth
-rw-r--r-- 1 root root 199271205 Oct 30 03:27 best_norm_ED.pth
-rw-r--r-- 1 root root       791 Oct 30 03:27 log_dataset.txt
-rw-r--r-- 1 root root    269588 Oct 30 03:27 log_train.txt
-rw-r--r-- 1 root root       921 Oct 30 03:27 opt.txt


In [5]:
from IPython.display import FileLink

FileLink(r'./saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth')

/kaggle/working/saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth